In [1]:
%reload_ext autoreload
%autoreload 2
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io.wavfile as wv
import scipy.signal as sig
import wave

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense

from util import util
from util import WavFileParts
from util.logUtil import LOG, LOG_HEADER
from util.confusionMatrix import ConfusionMatrix


Using TensorFlow backend.


### globale settings

In [2]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir
recordingDir = osDir + '/localizationRecordings'

if osDir == winDir:
    storageFolder = 'E:/SAA_DATA/storedData/'
else:
    storageFolder = '/Users/etto/Desktop/storedData/'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

orgsG428 = ['G428_0.0_1.4','G428_2.1_2.4']
orgsG527 = ['G527_0.5_1.4','G527_1.2_5.8']
orgsStudio = ['Studio_2.0_4.2','Studio_3.0_2.0']

NFFT = 1024

chunksBaseDir = 'chunks'
rooms = ['Studio', 'G428', 'G527']

### utility functies

In [3]:
def readSoundChunksDynamic(moduleString):
    chunks = importlib.import_module(moduleString).soundChunks
    wfPts = []
    for jsonString in chunks:
        wfPts.append(WavFileParts.WavFilePartFromJson(jsonString))
    return wfPts

def timeFunction(func):
    """
    Aanroep: bijv. fpc = timeFunction(lambda: getFilesPerCategory(srcDir))
    """
    startTime = datetime.now()
    print('Start: ' + startTime.strftime('%H:%M:%S') + '\n=================')

    res = func()
    
    endTime = datetime.now()
    print('\n=================\nEnd: ' + endTime.strftime('%H:%M:%S'))
    print('Time taken: '),
    print(endTime - startTime)
    print()
    
    return res
    
def storeTestData(allSpectros, allClasses, storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    df = pd.DataFrame(allSpectros)
    df.to_hdf(path_or_buf=filename, key='spectros_' + keyName)

    df = pd.DataFrame(allClasses)
    df.to_hdf(path_or_buf=filename, key='classes_' + keyName)

def retrieveTestData(storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    specDf = pd.read_hdf(path_or_buf=filename, key='spectros_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return specDf.values, classesDf.values
    

### functies tbv trainen

In [4]:
# Maakt een dictionary aan; per categorie alle files (volledig pad) uit de srcDir
# srcDir is een van de orgWavDirs, bijvoorbeeld
#    localizationFiles/20171025AllExtractionsMic4/G428_0.0_1.4
def getFilesPerCategory(srcDir):
    filesPerCategory = {}
    for catDirLong in glob.glob(srcDir + '/*'):
        catDir = catDirLong.replace('\\', '/')
        catDir = catDir.replace(srcDir + '/', '')

        filesPerCategory[catDir] = []
        for filename in glob.glob(catDirLong + '/*'):
            filename = filename.replace('\\','/')
            filesPerCategory[catDir].append(filename)
    return filesPerCategory

def getFilesPerCatFromMultipleDirs(srcDirs, srcDirsBase=''):
    filesPerCat = {}
    for dirName in srcDirs:
        srcDir = srcDirsBase + '/' + dirName
        fpcNw = getFilesPerCategory(srcDir)
        if not filesPerCat:
            filesPerCat = fpcNw
        else:
            for key in filesPerCat:
                filesPerCat[key] += fpcNw[key]
    return filesPerCat
        

In [5]:
# Maakt een dictionary aan; per categorie de spectrogrammen
def getSpectrosFromFilesPerCategory(filesPerCategory):
    spectros = {}
    for clz in classes:
        spectros[clz] = []
        for filename in filesPerCategory[clz]:
            fs, signal = wv.read(filename)
            freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
            spectros[clz].append(spectrogram.T)
    return spectros

In [6]:
def getClassLengths(spectrosPerCat):
    clzLengths = {}
    for clz in classes:
        clzLengths[clz] = sum([np.shape(lst)[0] for lst in spectrosPerCat[clz]])
    return clzLengths

In [7]:
# verwacht invoer van getSpectrosFromFilesPerCategory
# levert traindata op (X_train en Y_train)
def createTrainDataFromSpectros(spectrosPerCat, clzLengths):
    X_train = np.concatenate(spectrosPerCat[classes[0]], axis=0)
    for i in range(1, len(classes)):
        nwSpectros = np.concatenate(spectrosPerCat[classes[i]], axis=0)
        X_train = np.concatenate((X_train,nwSpectros), axis=0)
    
    # one-hot encoding voor Y_train
    nrFiles = clzLengths[classes[0]]
    Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

    nrFiles = clzLengths[classes[1]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.ones(nrFiles), np.zeros(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

    nrFiles = clzLengths[classes[2]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.zeros(nrFiles), np.ones(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
    
    return X_train, Y_train

### deep learning model

In [8]:
def create_model(layersizes):
    # create model
    model = Sequential()
    model.add(Dense(layersizes[0], input_dim=513, activation='relu'))
    for lsize in layersizes[1:]:
        model.add(Dense(lsize, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
def getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs):
    modelFilename = modelFilePath + baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    modelFilename += '.hd5'
    return modelFilename

In [10]:
def getModelName(baseModelFilename, layers, nrEpochs):
    modelFilename = baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    return modelFilename

In [11]:
def train_model(X_train, Y_train, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    history = timeFunction(lambda: soundModel.fit(X_train,Y_train, epochs=nrEpochs, shuffle=True, verbose=1))
    soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))
    plt.plot(history.history['loss'])
    plt.show()

In [12]:
def evaluate_model(X_test, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = load_model(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))

    # predicted classes
    predictions = soundModel.predict(X_test)
    predClasses = predictions.argmax(axis=1)

    matrix = ConfusionMatrix(classes)
    for vals in zip(realClasses, predClasses):
        matrix.add(int(vals[0]), int(vals[1]), 1)
    LOG(matrix.toString(),True)
    LOG('', True)
    LOG(matrix.toF1String(), True)
    return matrix

In [13]:
def train_and_evaluate_per_epoch(X_train, Y_train, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    for epNr in range(1, nrEpochs+1):
        LOG('\n*****************\n* Epoch nr {}\n*****************\n'.format(epNr), True)
        soundModel.fit(X_train,Y_train, epochs=1, shuffle=True, verbose=1)
        soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, epNr))
        evaluate_model(X_train, realClasses, layers, epNr, modelFilePath, baseModelFilename)

### functies tbv testen

In [14]:
def createAndStoreTestData(wavFileParts, baseDir, fileDate, micNr, storeFilename, keyName):
    allSpectros = np.array([])
    allClasses = np.array([])

    for wfPt in wavFileParts: #type: WavFilePart
        if not 'Gunshot' in wfPt.getSoundType():
            filename = baseDir + '/{:d}_{:d}_mono{:d}.wav'.format(fileDate, wfPt.fileNr, micNr)
            fs, signal = wv.read(filename)

            classNr = classes.index(wfPt.getSoundType().lower())
            for soundChunk in wfPt.getSoundChunks(micNr):
                startFrame = int(soundChunk[0] * fs)
                endFrame = int(soundChunk[1] * fs)

                sigChunk = signal[startFrame: endFrame]
                freq_array, segment_times, spectrogram = sig.spectrogram(x=sigChunk, fs=fs, nfft=NFFT, noverlap=0)
                if len(allSpectros) == 0:
                    allSpectros = spectrogram.T
                else:
                    allSpectros = np.append(allSpectros, spectrogram.T, axis=0)
                allClasses = np.append(allClasses, classNr * np.ones(len(segment_times)))
                
    storeTestData(allSpectros, allClasses, storeFilename, keyName)


In [15]:
def getNormalizedTrainDataFromFolders(orgWavDirs):
    fpc = getFilesPerCatFromMultipleDirs(orgWavDirs, baseSrcDir)
    spcs = getSpectrosFromFilesPerCategory(fpc)
    clzLengths = getClassLengths(spcs)
    X_train, Y_train = createTrainDataFromSpectros(spcs, clzLengths)
    # real train classes
    realTrainClasses = np.concatenate((np.zeros(clzLengths[classes[0]]), 
                                  np.ones(clzLengths[classes[1]]), 
                                  2*np.ones(clzLengths[classes[2]])))
    X_train = normalizeMatrix(X_train)
    return X_train, Y_train, realTrainClasses


In [16]:
def testModellenOnNormalizedSpecs(modelFilePath, layerss, micNr, testFileNames, showOverall=True):

    # test data
    keyname = 'mic{}'.format(micNr)

    for layers in layerss:
        for nrEpochs in nrsEpochs:
            overallMatrix = ConfusionMatrix(classes)
            for testFileName in testFileNames:
                testSpecs, testClasses = retrieveTestData(testFileName, keyname)
                testSpecs = normalizeMatrix(testSpecs)

                LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                           + '\n# ' + keyname
                           + '\n# ' + testFileName,
                           True)
                matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)
                overallMatrix.addMatrix(matrix.normalizedCopy())
            
            if showOverall:
                overallMatrix = overallMatrix.normalizedCopy()
                LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                           + '\n# ' + keyname
                           + '\n# ' + 'testData_Overall',
                           True)
                LOG(overallMatrix.toString(),True)
                LOG('', True)
                LOG(overallMatrix.toF1String(), True)

In [17]:
def normalizeMatrix(matrix):
    stdS = np.array([np.std(matrix, axis=1)])
#     varS = stdS**2
    meanS = np.array([np.mean(matrix, axis=1)])
    return (matrix - meanS.T) / stdS.T

### trainen van model obv: alle orgs, genormaliseerd

In [18]:
baseModelFilename = '20180822_allOrgs_norm'
modelFilePath = storageFolder
logPrefix = 'Alle Orgs Genormaliseerd'
orgWavDirs = orgWavDirs1 + orgWavDirs2
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

In [19]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getNormalizedTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Alle Orgs Genormaliseerd, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
1108305/1108305 [==============================] - 111s 100us/step - loss: 0.2824 - acc: 0.8890
                   m       v       e |  sens   acc
music         315794   41595    9147 |  0.86  0.92
voice          27350  329946    9427 |  0.90  0.92
environment     7160    5611  362275 |  0.97  0.97
--------------------------------------
prec            0.90    0.87    0.95

F1 overall: 0.91
F1 music: 0.88
F1 voice: 0.89
F1 environment: 0.96


*****************
* Epoch nr 2
*****************

Epoch 1/1
1108305/1108305 [==============================] - 110s 99us/step - loss: 0.2320 - acc: 0.9106
                   m       v       e |  sens   acc
music         346695   13745    6096 |  0.95  0.92
voice          60055  297985    8683 |  0.81  0.92
environment     8813    4560  361673 |  0.96  0.97
---------

In [20]:
# Test alle modellen
testModellenOnNormalizedSpecs(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)                

####################################
#
# 20180822_allOrgs_norm_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          18825    1449     365 |  0.91  0.94
voice            983   13119     334 |  0.91  0.94
environment       91      18   13116 |  0.99  0.98
--------------------------------------
prec            0.95    0.90    0.95

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.90
F1 environment: 0.97

####################################
#
# 20180822_allOrgs_norm_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8113     638     166 |  0.91  0.94
voice             39    2798      29 |  0.98  0.95
environment        7       0    2484 |  1.00  0.99
--------------------------------------
prec            0.99    0.81    0.93

F1 overall: 0.93
F1 music: 0.95
F1 voice: 0.89
F1 environment: 0.96

############################

                   m       v       e |  sens   acc
music           8232     545     140 |  0.92  0.94
voice            163    2698       5 |  0.94  0.95
environment        0       2    2489 |  1.00  0.99
--------------------------------------
prec            0.98    0.83    0.94

F1 overall: 0.94
F1 music: 0.95
F1 voice: 0.88
F1 environment: 0.97

####################################
#
# 20180822_allOrgs_norm_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          57114    5084    1397 |  0.90  0.93
voice           2826   34699     379 |  0.92  0.94
environment      409     328   41600 |  0.98  0.98
--------------------------------------
prec            0.95    0.87    0.96

F1 overall: 0.93
F1 music: 0.92
F1 voice: 0.89
F1 environment: 0.97

####################################
#
# 20180822_allOrgs_norm_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

       

####################################
#
# 20180822_allOrgs_norm_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          56793    5447    1355 |  0.89  0.94
voice           1190   36147     567 |  0.95  0.95
environment      411     141   41785 |  0.99  0.98
--------------------------------------
prec            0.97    0.87    0.96

F1 overall: 0.94
F1 music: 0.93
F1 voice: 0.91
F1 environment: 0.97

####################################
#
# 20180822_allOrgs_norm_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.91    0.07    0.02 |  0.91  0.96
voice           0.03    0.96    0.01 |  0.96  0.96
environment     0.00    0.00    0.99 |  0.99  0.99
--------------------------------------
prec            0.97    0.93    0.97

F1 overall: 0.95
F1 music: 0.94
F1 voice: 0.94
F1 enviro

### G428

In [21]:
baseModelFilename = '20180822_G428model_norm'
modelFilePath = storageFolder
logPrefix = 'G428model Genormaliseerd'
orgWavDirs = orgsG428
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

In [22]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getNormalizedTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# G428model Genormaliseerd, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
375667/375667 [==============================] - 50s 134us/step - loss: 0.2993 - acc: 0.8797
                   m       v       e |  sens   acc
music          96013   22248    3080 |  0.79  0.91
voice           7217  112784    1632 |  0.93  0.91
environment     1344    2401  128948 |  0.97  0.98
--------------------------------------
prec            0.92    0.82    0.96

F1 overall: 0.90
F1 music: 0.85
F1 voice: 0.87
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
375667/375667 [==============================] - 48s 126us/step - loss: 0.2360 - acc: 0.9071
                   m       v       e |  sens   acc
music         109841    9531    1969 |  0.91  0.92
voice          15295  104624    1714 |  0.86  0.93
environment     1885    1458  129350 |  0.97  0.98
--------------

In [23]:
# Test alle modellen
testModellenOnNormalizedSpecs(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)                

####################################
#
# 20180822_G428model_norm_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          17142    2998     499 |  0.83  0.91
voice            594   13627     215 |  0.94  0.92
environment       65      17   13143 |  0.99  0.98
--------------------------------------
prec            0.96    0.82    0.95

F1 overall: 0.91
F1 music: 0.89
F1 voice: 0.88
F1 environment: 0.97

####################################
#
# 20180822_G428model_norm_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           6928    1648     341 |  0.78  0.86
voice             21    2830      15 |  0.99  0.88
environment        7       4    2480 |  1.00  0.97
--------------------------------------
prec            1.00    0.63    0.87

F1 overall: 0.86
F1 music: 0.87
F1 voice: 0.77
F1 environment: 0.93

########################

                   m       v       e |  sens   acc
music           7779     958     180 |  0.87  0.91
voice            140    2712      14 |  0.95  0.92
environment        0       4    2487 |  1.00  0.99
--------------------------------------
prec            0.98    0.74    0.93

F1 overall: 0.91
F1 music: 0.92
F1 voice: 0.83
F1 environment: 0.96

####################################
#
# 20180822_G428model_norm_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          53570    7902    2123 |  0.84  0.91
voice           2247   35138     519 |  0.93  0.92
environment      532     325   41480 |  0.98  0.98
--------------------------------------
prec            0.95    0.81    0.94

F1 overall: 0.91
F1 music: 0.89
F1 voice: 0.86
F1 environment: 0.96

####################################
#
# 20180822_G428model_norm_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

   

####################################
#
# 20180822_G428model_norm_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          54667    7262    1666 |  0.86  0.92
voice           2259   35310     335 |  0.93  0.93
environment      592     813   40932 |  0.97  0.98
--------------------------------------
prec            0.95    0.81    0.95

F1 overall: 0.91
F1 music: 0.90
F1 voice: 0.87
F1 environment: 0.96

####################################
#
# 20180822_G428model_norm_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.89    0.10    0.02 |  0.89  0.94
voice           0.05    0.94    0.01 |  0.94  0.94
environment     0.01    0.02    0.97 |  0.97  0.98
--------------------------------------
prec            0.93    0.89    0.97

F1 overall: 0.93
F1 music: 0.91
F1 voice: 0.91
F1 en

### G527

In [24]:
baseModelFilename = '20180822_G527model_norm'
modelFilePath = storageFolder
logPrefix = 'G527model Genormaliseerd'
orgWavDirs = orgsG527
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

In [25]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getNormalizedTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# G527model Genormaliseerd, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
378662/378662 [==============================] - 65s 171us/step - loss: 0.2897 - acc: 0.8847
                   m       v       e |  sens   acc
music         104860   14119    3326 |  0.86  0.92
voice          10072  108106    4436 |  0.88  0.92
environment     2081    1558  130104 |  0.97  0.97
--------------------------------------
prec            0.90    0.87    0.94

F1 overall: 0.90
F1 music: 0.88
F1 voice: 0.88
F1 environment: 0.96


*****************
* Epoch nr 2
*****************

Epoch 1/1
378662/378662 [==============================] - 59s 156us/step - loss: 0.2352 - acc: 0.9077
                   m       v       e |  sens   acc
music         105684   13526    3095 |  0.86  0.93
voice           9313  110414    2887 |  0.90  0.93
environment     1870    1916  129957 |  0.97  0.97
--------------

In [26]:
# Test alle modellen
testModellenOnNormalizedSpecs(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)                

####################################
#
# 20180822_G527model_norm_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          17274    2617     748 |  0.84  0.90
voice           1332   12551     553 |  0.87  0.91
environment      145      11   13069 |  0.99  0.97
--------------------------------------
prec            0.92    0.83    0.91

F1 overall: 0.89
F1 music: 0.88
F1 voice: 0.85
F1 environment: 0.95

####################################
#
# 20180822_G527model_norm_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           7245    1324     348 |  0.81  0.87
voice            139    2649      78 |  0.92  0.89
environment        5       2    2484 |  1.00  0.97
--------------------------------------
prec            0.98    0.67    0.85

F1 overall: 0.86
F1 music: 0.89
F1 voice: 0.77
F1 environment: 0.92

########################

                   m       v       e |  sens   acc
music           7736     929     252 |  0.87  0.90
voice            234    2612      20 |  0.91  0.92
environment        6       5    2480 |  1.00  0.98
--------------------------------------
prec            0.97    0.74    0.90

F1 overall: 0.89
F1 music: 0.92
F1 voice: 0.81
F1 environment: 0.95

####################################
#
# 20180822_G527model_norm_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          51752    8315    3528 |  0.81  0.89
voice           3947   32905    1052 |  0.87  0.91
environment      579     302   41456 |  0.98  0.96
--------------------------------------
prec            0.92    0.79    0.90

F1 overall: 0.88
F1 music: 0.86
F1 voice: 0.83
F1 environment: 0.94

####################################
#
# 20180822_G527model_norm_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################

   

####################################
#
# 20180822_G527model_norm_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          50933    9657    3005 |  0.80  0.90
voice           1365   35608     931 |  0.94  0.91
environment      584     351   41402 |  0.98  0.97
--------------------------------------
prec            0.96    0.78    0.91

F1 overall: 0.89
F1 music: 0.87
F1 voice: 0.85
F1 environment: 0.94

####################################
#
# 20180822_G527model_norm_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.84    0.12    0.03 |  0.84  0.93
voice           0.03    0.95    0.02 |  0.95  0.94
environment     0.01    0.00    0.99 |  0.99  0.98
--------------------------------------
prec            0.95    0.88    0.95

F1 overall: 0.93
F1 music: 0.89
F1 voice: 0.91
F1 en

### Studio

In [27]:
baseModelFilename = '20180822_StudioModel_norm'
modelFilePath = storageFolder
logPrefix = 'Studio model Genormaliseerd'
orgWavDirs = orgsStudio
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 3
nrsEpochs = [1,2,3]

In [28]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getNormalizedTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Studio model Genormaliseerd, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
353976/353976 [==============================] - 81s 229us/step - loss: 0.3150 - acc: 0.8763
                   m       v       e |  sens   acc
music         103228   17537    2125 |  0.84  0.92
voice           8034  110453    3989 |  0.90  0.91
environment     2096    2092  104422 |  0.96  0.97
--------------------------------------
prec            0.91    0.85    0.94

F1 overall: 0.90
F1 music: 0.87
F1 voice: 0.87
F1 environment: 0.95


*****************
* Epoch nr 2
*****************

Epoch 1/1
353976/353976 [==============================] - 75s 212us/step - loss: 0.2489 - acc: 0.9032
                   m       v       e |  sens   acc
music         112861    8446    1583 |  0.92  0.93
voice          14019  105303    3154 |  0.86  0.92
environment     2287    2031  104292 |  0.96  0.97
-----------

In [29]:
# Test alle modellen
testModellenOnNormalizedSpecs(modelFilePath=storageFolder, 
             layerss=layersList, 
             micNr=4, 
             testFileNames=['testData_G428', 'testData_G527', 'testData_Studio'], 
             showOverall=True)                

####################################
#
# 20180822_StudioModel_norm_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          17859    2545     235 |  0.87  0.92
voice            845   13240     351 |  0.92  0.92
environment      153      64   13008 |  0.98  0.98
--------------------------------------
prec            0.95    0.84    0.96

F1 overall: 0.92
F1 music: 0.90
F1 voice: 0.87
F1 environment: 0.97

####################################
#
# 20180822_StudioModel_norm_100_20ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           7784    1051      82 |  0.87  0.92
voice             47    2783      36 |  0.97  0.92
environment       11       2    2478 |  0.99  0.99
--------------------------------------
prec            0.99    0.73    0.95

F1 overall: 0.91
F1 music: 0.93
F1 voice: 0.83
F1 environment: 0.97

####################

                   m       v       e |  sens   acc
music           8437     397      83 |  0.95  0.95
voice            187    2669      10 |  0.93  0.96
environment        5       7    2479 |  1.00  0.99
--------------------------------------
prec            0.98    0.87    0.96

F1 overall: 0.95
F1 music: 0.96
F1 voice: 0.90
F1 environment: 0.98

####################################
#
# 20180822_StudioModel_norm_400_250_100_20ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          58656    3863    1076 |  0.92  0.94
voice           3715   33655     534 |  0.89  0.94
environment      613     180   41544 |  0.98  0.98
--------------------------------------
prec            0.93    0.89    0.96

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.89
F1 environment: 0.97

####################################
#
# 20180822_StudioModel_norm_400_250_100_20ep2
# mic4
# testData_Overall
#
####################################


####################################
#
# 20180822_StudioModel_norm_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          58909    3447    1239 |  0.93  0.94
voice           3421   34036     447 |  0.90  0.95
environment      461     516   41360 |  0.98  0.98
--------------------------------------
prec            0.94    0.90    0.96

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.90
F1 environment: 0.97

####################################
#
# 20180822_StudioModel_norm_400_300_200_100_50_20_10ep3
# mic4
# testData_Overall
#
####################################

                   m       v       e |  sens   acc
music           0.94    0.04    0.01 |  0.94  0.95
voice           0.09    0.90    0.01 |  0.90  0.95
environment     0.01    0.01    0.99 |  0.99  0.99
--------------------------------------
prec            0.90    0.94    0.98

F1 overall: 0.94
F1 music: 0.92
F1 voice: 0.92
F